In [1]:
from elasticsearch import Elasticsearch
import os
import json
import uuid
from elasticsearch import helpers

In [2]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}], request_timeout=30)
if es.ping():
    print('Connected to ES!')
else:
    print('Could not connect!')
       

Connected to ES!


In [3]:

def script_path():
    path = os.path.dirname(os.path.realpath("data_trials_new.json"))
    if os.name == 'posix': # posix is for macOS or Linux
        path = path + "/"
    else:
        path = path + chr(92) # backslash is for Windows
    return path


'''
this function opens a file and returns its
contents as a list of strings split by linebreaks
'''
def get_data_from_file(self, path=script_path()):
    file = open(path + str(self), encoding="utf8", errors='ignore')
    data = [line.strip() for line in file]
    file.close()
    return data

'''
generator to push bulk data from a JSON
file into an Elasticsearch index
'''
def bulk_json_data(json_file, _index, doc_type):
    json_list = get_data_from_file(json_file)
    for doc in json_list:
    # use a `yield` generator so that the data
    # isn't loaded into memory
        if '{"index"' not in doc:
            yield {
                "_index": _index,
                "_type": doc_type,
                "_source": doc
            }


In [4]:
import os
import json
import uuid
from elasticsearch import helpers
try:
    # make the bulk call, and get a response
    response = helpers.bulk(es, bulk_json_data("data_trials_new.json", "stackoverflow3", "people"))

    #response = helpers.bulk(elastic, actions, index='employees', doc_type='people')
    print ("\nRESPONSE:", response)
except Exception as e:
    print("\nERROR:", e)


RESPONSE: (50, [])


C:\Users\swapnil\installations\anaconda\anaconda\lib\site-packages\elasticsearch\connection\base.py:193: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [4]:
res = es.search(index="test-database1", body={"query": {"match_all": {}}})
res


{'took': 1144,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 50, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'test-database1',
    '_type': '_doc',
    '_id': 'JS5K63YBXX3FrV-li1Ge',
    '_score': 1.0,
    '_source': {'question': 'SQL select only rows with max value on a column [duplicate]',
     'details': "I have this table for documents (simplified version here):+------+-------+--------------------------------------+\n| id   | rev   | content                              |\n+------+-------+--------------------------------------+\n| 1    | 1     | ...                                  |\n| 2    | 1     | ...                                  |\n| 1    | 2     | ...                                  |\n| 1    | 3     | ...                                  |\n+------+-------+--------------------------------------+\nHow do I select one row per id and only the greatest rev?\nWith the above data, the r

In [10]:
es.search(index="stackoverflow3", body={"query": {"match": {'question': 'How to add JTable in JPanel with null layout?'}}})


{'took': 14,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 203, 'relation': 'eq'},
  'max_score': 26.255737,
  'hits': [{'_index': 'stackoverflow3',
    '_type': 'people',
    '_id': 'w8-N53YBnF2Y8Y0cxzN6',
    '_score': 26.255737,
    '_source': {'question': 'How to add JTable in JPanel with null layout?',
     'details': 'I want to add JTable into JPanel whose layout is null.  JPanel contains other components. I have to add JTable at proper position.',
     'answers': ['I want to add JTable into JPanel whose layout is null.  JPanel contains other components. I have to add JTable at proper position.',
      'The Java Tutorial has comprehensive information on using layout managers. See the  lesson for further details.One aspect of layouts that is  covered well by the tutorial is that of nested layouts, putting one layout inside another to get complex effects.The following code puts a variety of components into a 

In [8]:
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [4]:
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
embed = hub.load('universal_encoder')
def make_vector(query):
    embeddings = embed([query])
    vector = []
    for i in embeddings[0]:
        vector.append(float(i))
    return vector

In [5]:
def search(query):
    def norm_list(lis):
        scores = [x[0] for x in lis]
        ma = max(scores)
        mi = min(scores)
        for i in range(len(lis)):
            lis[i][0] = (lis[i][0] - mi)/(ma - mi)
        return lis
    
    request={
            'query':{ 'match':{"question":query } }
            }

    res= es.search(index='test-database1',body=request)
    l1 = []
    for hit in res['hits']['hits']:
        l1.append([hit['_score'] , hit['_id']])
# change the cosine similarity to euclidean distance

    query_vector = make_vector(query)
    request = {"query" : {
                "script_score" : {
                    "query" : {
                        "match_all": {}
                    },
                    "script" : {
                        "source": "cosineSimilarity(params.query_vector, 'total_vectors') + 1.0",
                        "params": {"query_vector": query_vector}
                    }
                }
             }
    }

    res= es.search(index='test-database1',body=request)
    l2 = []
    for hit in res['hits']['hits']:
        l2.append([hit['_score'] , hit['_id']])
    
    l1 = norm_list(l1)
    l2 = norm_list(l2)
    
    # getting the weighted average score for the text search and semantics search
    temp_doc = {}
    for i in l1:
        temp_doc[i[1]]  = i[0]*2
    for i in l2:
        temp_doc[i[1]] = temp_doc.get(i[1] , 0) + i[0]*5
    
    inverse_temp_doc = [(i[1] , i[0])  for i in temp_doc.items()]
    inverse_temp_doc = sorted(inverse_temp_doc , reverse = True)
    return inverse_temp_doc[:10]

In [30]:
for i in search(" Sql  [duplicate] "):
    result = es.search(index="test-database1",body={"query": {
    "terms": {
      "_id": ['{}'.format(i[1])]
    }
  }})
    for x in result['hits']['hits']:
        print (x['_source']['question'])
        print (x['_source']['details'])
        print (x['_source']['answers'])

    

SQL select only rows with max value on a column [duplicate]
I have this table for documents (simplified version here):+------+-------+--------------------------------------+
| id   | rev   | content                              |
+------+-------+--------------------------------------+
| 1    | 1     | ...                                  |
| 2    | 1     | ...                                  |
| 1    | 2     | ...                                  |
| 1    | 3     | ...                                  |
+------+-------+--------------------------------------+
How do I select one row per id and only the greatest rev?
With the above data, the result should contain two rows: [1, 3, ...] and [2, 1, ..]. I'm using .Currently I use checks in the while loop to detect and over-write old revs from the resultset. But is this the only method to achieve the result? Isn't there a  solution?
As the answers suggest, there  a SQL solution, and . 
I noticed after adding the above , the rate at which th